In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Parameters

modes = ['vae','classall','classred','syllall','syllred','phonall','phonred','siamese'] # Triplet!!
#clfs = ['slp','mlp','logr','knn','rf','xgboost']#
clfs = ['knn']

list_test_participants_avp = [8,10,18,23]
list_test_participants_lvt = [0,6,7,13]

# Calculate utterance-wise weights

num_test_utterances = []
for part in list_test_participants_avp:
    if part<=9:
        test_dataset = np.load('../../data/interim/AVP/Dataset_Test_0' + str(part) + '.npy')
    else:
        test_dataset = np.load('../../data/interim/AVP/Dataset_Test_' + str(part) + '.npy')
    num_test_utterances.append(test_dataset.shape[0])
for part in list_test_participants_lvt:
    if part<=9:
        test_dataset = np.load('../../data/interim/LVT/Dataset_Test_0' + str(part) + '.npy')
    else:
        test_dataset = np.load('../../data/interim/LVT/Dataset_Test_' + str(part) + '.npy')
    num_test_utterances.append(test_dataset.shape[0])
utterance_wise_weights = num_test_utterances/np.sum(np.array(num_test_utterances))

In [3]:
# Results

for b in range(len(modes)):

    for c in range(len(clfs)):

        mode = modes[b]
        clf = clfs[c]

        accuracies_raw = np.load('../../results/' + mode + '/accuracies.npy')
        accuracies_mean = np.mean(np.mean(np.mean(accuracies_raw,axis=-1),axis=-1),axis=-1)
        accuracies_std = np.std(np.mean(np.mean(accuracies_raw,axis=-1),axis=-1),axis=-1)
        
        print([mode,clf])
        print([accuracies_mean[0],accuracies_std[0]])

['vae', 'knn']
[0.7910309827280464, 0.01624331950467861]
['class', 'knn']
[0.7988555147232376, 0.022928629687180183]
['syllall', 'knn']
[0.8896069759332862, 0.014638511095622024]
['syllred', 'knn']
[0.8844865828108878, 0.02379058000055109]
['phonall', 'knn']
[0.8765230289629609, 0.01855451173206599]
['phonred', 'knn']
[0.8635369357898576, 0.015212323426247776]
['siamese', 'knn']
[0.8642709503450103, 0.025331657417954904]


In [8]:
# Results Utterance-Wise

for b in range(len(modes)):

    for c in range(len(clfs)):

        mode = modes[b]
        clf = clfs[c]

        accuracies_raw = np.load('../../results/' + mode + '/accuracies.npy')

        for i in range(accuracies_raw.shape[0]):
            for j in range(accuracies_raw.shape[1]):
                for k in range(accuracies_raw.shape[2]):
                    accuracies_raw[i,j,k] *= utterance_wise_weights*8

        accuracies_mean = np.mean(np.mean(np.mean(accuracies_raw,axis=-1),axis=-1),axis=-1)
        accuracies_std = np.std(np.mean(np.mean(accuracies_raw,axis=-1),axis=-1),axis=-1)
        
        print([mode,clf])
        print([accuracies_mean[0],accuracies_std[0]])

['eng_mfcc_env', 'knn']
[0.810702875399361, 0.0]
['vae', 'knn']
[0.7723642172523961, 0.024009536836821766]
['class', 'knn']
[0.7736421725239616, 0.015443244777772473]
['syllall', 'knn']
[0.8630990415335462, 0.017131424180562703]
['syllred', 'knn']
[0.859185303514377, 0.03127086060771459]
['phonall', 'knn']
[0.8460063897763577, 0.024275894840729593]
['phonred', 'knn']
[0.8334664536741213, 0.02157806402161614]
['siamese', 'knn']
[0.8297124600638976, 0.032210985573016736]
